## Using the Workforce Python Module to Automate Workforce for ArcGIS

Workforce for ArcGIS is a mobile solution that uses the power of location-based decision making for better field workforce coordination and teamwork. It is composed of a web app used by project administrators and dispatchers in the office, and a mobile app used by mobile workers on their devices. Organizations using Workforce for ArcGIS get these benefits:

- Everything you need on one device—Mobile workers can easily view and process work assignments, provide updates on work status, and inform others of their location, all from one device.

- Greater agility—Using real-time and location-based information, dispatchers can assign and prioritize fieldwork on the fly and ensure that work is assigned to the right people at the right time.

- Increased productivity—Replace time-consuming and error-prone manual workforce management processes, reduce downtime, and keep projects on schedule.

### Workforce Schema

**Note**: The following was extracted/summarized from [here](https://doc.arcgis.com/en/workforce/android-phone/help/workforce-schema.htm).

A workforce project is composed of four feature layers and four coded value domains with a predefined schema. The name of each feature layer is a combination of a moniker, describing the purpose of the feature layer, appended with the GUID of the Workforce project item. For example, the Workers layer associated with a project with GUID 5dd018fcd88c4d33814cf3da9c44061e would be named workers_5dd018fcd88c4d33814cf3da9c44061e. This guarantees uniqueness of each feature layer.

The four feature layers are as follows:

- **Workers**

  - A point feature layer that contains a record for each mobile worker who is included in the project.
  - Includes information about the mobile worker, including their contact number and job title.
  - The mobile worker's ArcGIS organizational user name is stored in the userId field.
  - The layer tracks who created and last updated each mobile worker.
  - There is a primary key-foreign key (PK-FK) relationship from OBJECTID to Assignments.workerId. Using the OBJECTID value from the Workers layer as the Assignments.workerId field value associates the mobile worker with all their assignments.
  - The layer has the following coded value domain associations:
  - The status field is associated with the Worker_Status coded value domain to track the mobile worker status.

- **Assignments**

  - A point feature layer that contains a record for each assignment.
  - Includes information about the assignment, including its status, location, and description, among others.
  - The layer tracks who created and last updated each assignment.
  - Attachments are enabled on the feature layer.
  - The layer contains foreign keys for some fields, associating values from another layer with this layer:
    - Assignments.workerId to Workers.OBJECTID.
    - Assignments.dispatcherId to Dispatchers.OBJECTID.
    - Assignments.workOrderId can be used as a foreign key to an external system, such as an asset or maintenance management system, by providing values from the other system.
  - The layer has the following coded value domain associations:
    - The status field is associated with the Assign_Status coded value domain to track the assignment status.
    - The priority field is associated with the Priority coded value domain to manage the priority of work assignments.
    - The assignmentType field is associated with the Assign_Type coded value domain to store the assignment types for the project.

- **Dispatchers**
  - A point feature layer that contains a record for each dispatcher within the project.
  - Includes information about the dispatcher, including their name and contact number.
  - The dispatcher's ArcGIS organizational user name is stored in the userId field.
  - The layer tracks who created and last updated each dispatcher.
  - There is a PK-FK relationship from OBJECTID to Assignments.dispatcherId. Using the OBJECTID value from the Dispatchers layer as the Assignments.dispatcherId field value associates the dispatcher with all the assignments they assigned.

- **Location Tracking**
  - A point feature layer that contains a record for each point location logged while location tracking is enabled.
  - The layer tracks who created and last updated each location track.
  
Additionally, a workforce project contains two webmaps:

- **Dispatcher Webmap**
  - This map is what the dispatchers using the back-office web app see
  - It shows the assignments and worker locations
  - Additional layers can be added to this map
  
- **Worker Webmap**
  - This map is what a field worker uses on their iOS or Android device
  - Additional layers can be added to this map
  
When a new project is created via the web application, a new **Group** is created. All of the layers, webmaps, and the project item itself are shared into this group.

When a new project is created via the web application, a new **Folder** is created. All of the layers, webmaps, and the project item itself are shared into this folder.

Finally, there is the actual **Workforce Project** item. This is an item on the portal that stores project meta data in json format.

```json
{
    "workerWebMapId": "<worker-map-id>",
    "dispatcherWebMapId": "<dispatcher-map-id>",
    "dispatchers": {
        "serviceItemId": "<item-id>",
        "url": "<layer-url>"
    },
    "assignments": {
        "serviceItemId": "<item-id>",
        "url": "<layer-url>"
    },
    "workers": {
        "serviceItemId": "<item-id>",
        "url": "<layer-url>"
    },
    "tracks": {
        "serviceItemId": "<item-id>",
        "url": "<layer-url>",
        "enabled": <true | false>,
        "updateInterval": 30
    },
    "version": "1.2.0",
    "groupId": "<group-id>",
    "folderId": "<folder-d>",
    "assignmentIntegrations": [
        {
            "id": "default-navigator",
            "prompt": "Navigate to Assignment",
            "urlTemplate": "arcgis-navigator://?stop=${assignment.latitude},${assignment.longitude}&stopname=${assignment.location}&callback=arcgis-workforce://&callbackprompt=Workforce"
        }
    ]
}
```


### Common tasks that can be accomplished with this module

#### Workers and Dispatchers
- Adding Dispatchers and Workers to a Project
- Removing Dispatchers and Workers from a Project
- Updating Workers and Dispatchers in a Project
- Querying Workers and Dispatchers in a Project

#### Assignments
- Adding Assignments to a Project
- Removing Assignments from a Project
- Updating Assignments in a Project
- Assigning Assignments in a Project
- Querying Assignments in a Project
- Adding/Removing/Downloading Attachments

#### Tracks
- Querying Tracks (for analysis)

### Getting Started

A user must be logged on to a GIS in order to fetch a Project. The workforce functionality is available in `arcgis.apps.workforce`

In [1]:
from arcgis.gis import GIS
from arcgis.apps import workforce

gis = GIS('https://arcgis.com', 'workforce_scripts', 'esri12345')

### Accessing a project

In [2]:
item = gis.content.get("29eec3c45982458d876c5e1f2f32333d")
project = workforce.Project(item)

#### Accessing assignments in a project

In [3]:
# Uses an AssignmentManager to perform CRUD operations (with significatnt validation) for assignments
assignments = project.assignments.query()

# Let's view the first assignment
assignment = assignments[0]
print(f"Status: {assignment.status.name}")
print(f"Description: {assignment.description}")
print(f"Priority: {assignment.priority.name}")
print(f"Assigned To: {assignment.worker.name}")
print(f"Type: {assignment.assignment_type}")

# Let's update the description
assignment.description = "You need to do an inspection here"
project.assignments.update([assignment])
print("--------------------")
print(f"Updated Description: {project.assignments.query()[0].description}")

# Let's download the attachments for this assignment using the AssignmentAttachmentManager
assignment.attachments.download()

Status: ASSIGNED
Description: You need to do an inspection here
Priority: MEDIUM
Assigned To: Aaron Pulver
Type: Inspection
--------------------
Updated Description: You need to do an inspection here


['/Users/aaro8157/PycharmProjects/geosaurus/examples/Apps/esri_logo1.png']

#### Accessing assignment types in a project

In [4]:
# Uses an AssignmentTypeManager to perform CRUD operations to the assignment types
assignment_types = project.assignment_types.get_all()
for at in assignment_types:
    print(f"Type: {at.name}")
    
# Let's add a new assignment type
assignment_type = workforce.AssignmentType(project)
assignment_type.name = "Repair"
project.assignment_types.add([assignment_type])

# Let's confirm that it was added
print("--------------------")
assignment_types = project.assignment_types.get_all()
for at in assignment_types:
    print(f"Type: {at.name}")

Type: Inspection
Type: Removal
--------------------
Type: Inspection
Type: Removal
Type: Repair


#### Accessing workers in a project

In [5]:
# Uses a WorkerManager to perform CRUD operations 
workers = project.workers.query()
worker = workers[0]
print(f"Name: {worker.name}")
print(f"Number: {worker.contact_number}")
    
# Let's update the workers number
worker.contact_number = "123-456-7890"
project.workers.update([worker])
print("--------------------")
print(f"Number: {project.workers.query()[0].contact_number}")

Name: Aaron Pulver
Number: None
--------------------
Number: 123-456-7890


In [6]:
# Let's add a new worker
worker = workforce.Worker(project)
worker.name = "Demo User"
worker.user_id = "demouser_nitro"
worker.contact_number = "123-987-4560"
project.workers.add([worker])

#### Accessing dispatchers in a project

In [7]:
# Uses a DispatcherManager to perform CRUD operations 
dispatchers = project.dispatchers.query()
dispatcher = dispatchers[0]
print(f"Name: {dispatcher.name}")
print(f"Number: {dispatcher.contact_number}")
    
# Let's update the dispatchers number
dispatcher.contact_number = "123-456-7890"
project.dispatchers.update([dispatcher])
print("--------------------")
print(f"Number: {project.dispatchers.query()[0].contact_number}")

Name: workforce scripts
Number: 987-654-3210
--------------------
Number: 123-456-7890


#### Accessing the webmaps

In [21]:
# worker webmap
project.worker_webmap
# dispatcher webmap
project.dispatcher_webmap

### Putting it all together (and highlighting validation)

In [9]:
# Let's add a new assignment and assign it to demouser
from datetime import datetime
demouser = project.workers.get(user_id='demouser_nitro')
dispatcher = project.dispatchers.get(user_id='workforce_scripts')
repair = project.assignment_types.get(name="Repair")
new_assignment = workforce.Assignment(project)
new_assignment.assignment_type = repair
new_assignment.status = workforce.AssignmentStatus.ASSIGNED
new_assignment.assigned_date = datetime.now()
new_assignment.worker = demouser
new_assignment.dispatcher = dispatcher

# Let's try to add a new assignment now (should get validation error)
try:
    project.assignments.add([new_assignment])
except Exception as e:
    print(e.message)

Assignment cannot have an empty location


In [10]:
# Okay so we need to add a location
new_assignment.location = "ESRI, Redlands, CA"

# Let's try to add a new assignment now (should get validation error)
try:
    project.assignments.add([new_assignment])
except Exception as e:
    print(e.message)

Assignment must have geometry


In [11]:
# Let's use the geocoder to add some geometry
from arcgis.geocoding import geocode
new_assignment.geometry = geocode(new_assignment.location, out_sr=102100)[0]['location']

# Let's try to add a new assignment now
project.assignments.add([new_assignment])

[{"attributes": {"status": 1, "priority": 0, "assignmentType": 3, "assignedDate": 1521222078940.6199, "workerId": 7, "dispatcherId": 1, "location": "ESRI, Redlands, CA", "OBJECTID": 6, "GlobalID": "ED481ADD-7B1F-4CB1-814F-6D367DE7B965"}, "geometry": {"x": -13046163.418234022, "y": 4036389.770988603}}]

In [12]:
# Show the new assignment in Redlands
project.dispatcher_webmap

### Reset the Demo Project

In [13]:
project.assignments.remove(project.assignments.query(where='assignmentType=3'))
project.assignment_types.remove([project.assignment_types.get(name="Repair")])
project.workers.remove([project.workers.get(user_id="demouser_nitro")])
a = project.assignments.get(object_id=1)
a.description = "Do some work at the ESRI R&D Center"
project.assignments.update([a])

w1 = project.workers.get(object_id=1)
w1.contact_number = None
project.workers.update([w1])
d1 = project.dispatchers.get(object_id=1)
d1.contact_number = "987-654-3210"
project.dispatchers.update([d1])